# Interactive Map - Folium (Leaflet)

For experimentation on creating an interactive map with our data.

In [1]:
### Import Libraries

# File manipulation

import os # For working with Operating System
from sys import platform # Diagnose operating system
import json # For working with Json files
import requests # Processing online requests


# Analysis

import numpy as np # For working with Arrays
import pandas as pd # Data Manipulation
import geopandas as gpd # Spatial Data Manipulation

# Visualization

import folium # Interactive Leaflet.js mapping 
from folium import features

import warnings
warnings.filterwarnings('ignore') # Ignores some warnings

In [2]:
### Definitions

cwd = os.getcwd() # Current Working Directory

# Forward or back slashs for filepaths? <- Not sure here. Only know Windows & Linux

if platform == "linux" or platform == "linux2":
    slash = '/'
elif platform == 'win32':
    slash = '\\'

# Load Data

In [3]:
# Load Data -  We'll have to convert all data to another projection... <- Maybe go back and do this in the cleaning notebooks
# Let's use WGS84 EPSG:4326...

# Get GeoJsons from GitHub

url = (
    "https://raw.githubusercontent.com/RwHendrickson/MappingGZ/main/Prototype/Notebooks/CleaningData"
)
mpls_boundary_path = f"{url}/Boundary/mpls_boundaryWGS84.geojson"

# Get bigger files from computer

cwd = os.getcwd()

mpls_aadt_path = cwd + slash + 'CleaningData' + slash + 'Traffic' + slash + 'mpls_aadtWGS84.geojson'
mpls_emissions_path = cwd + slash + 'CleaningData' + slash + 'PermittedEmissions' + slash + 'mpls_emissions.csv'

# Load files as geodataframes for focusing data

traffic = gpd.read_file(mpls_aadt_path)
emissions = pd.read_csv(mpls_emissions_path)

emissions_geo = gpd.GeoDataFrame(emissions, geometry = gpd.points_from_xy(emissions.LONGITUDE, emissions.LATITUDE, crs = 'EPSG:4326'))

In [4]:
# Focus Data

## Traffic

aadt = traffic[['SEQUENCE_N', 'ROUTE_LABE', 'CURRENT_VO', 'geometry']]

# Emissions

emissions_2020 = emissions_geo[emissions_geo['YEAR']==2020][['FACILITY_NAME', 'INDUSTRY_TYPE', 'NAICS_CODE', 'POLLUTANT', 'LBS_EMITTED','geometry']]
vocs = emissions_2020[emissions_2020['POLLUTANT'] == 'Volatile Organic Compounds']
# pm = emissions_2020[emissions_2020['POLLUTANT'] == 'PM Primary']

# Define Visualizations

## Annual Average Daily Traffic

In [5]:
### AADT style

def style_aadt(feature):
    '''Set Thickness/color of streets to be proportional to traffic volume'''
    
    volume = feature['properties']['CURRENT_VO']
    
    
    # This is not the way to add a popup
    popup = f"Road: {feature['properties']['ROUTE_LABE']}\n Current Average Vehicles per Day: {feature['properties']['CURRENT_VO']}" 
    
    if volume < 1773: # Low volume
        return {
        "weight": 1,
        "color": "#848484",
        'popup': popup
    }
    elif volume < 5378: # Low-mid volume
        return {
        "weight": 2,
        "color": "#936d6d",
        'popup': popup
    }
    elif volume < 16308: # Mid volume
        return {
        "weight": 3,
        "color": "#a94646",
        'popup': popup
    }
    elif volume < 100000: # High Volume
        return {
        "weight": 5,
        "color": "#f90707",
        'popup': popup
    }
    else: # Very High Volume Potentially could use another break here
        return {
        "weight": 10,
        "color": "#090707",
        'popup': popup
    }
    

## Permitted Volatile Organic Compound Emissions

In [6]:
    
# Plot the facilities' VOC emission
    
def plot_voc(point):
    '''plot circle of facilities related to amount of vocs emitted'''    
    
    lbs = point.LBS_EMITTED 
    
    if lbs < 25: # Low volume
        weight = 1,
        color = '#848484'
    elif lbs < 100: # Low-mid volume
        weight = 3,
        color = '#936d6d'
    elif lbs < 1000: # Mid volume
        weight = 5,
        color = '#a94646'
    elif lbs < 10000: # High Volume
        weight = 10,
        color = '#f90707'
    else: # Very High Volume
        weight = 15,
        color = '#090707'
        
    folium.Circle(location=[point.geometry.y,point.geometry.x],
                       weight = weight,
                       color = color,
                       popup = f'Facility: {point.FACILITY_NAME}\n Lbs of VOCs Emitted in 2020: {lbs}\n Industry Type: {point.INDUSTRY_TYPE}\n NAICS code: {point.NAICS_CODE}'
                       ).add_to(m)

# Add to the map!

In [7]:
# Make a basic folium map

m = folium.Map(location=[44.986656, -93.258133],
               zoom_start=12,
               tiles = 'cartodbpositron')

# Add boundary

folium.GeoJson(mpls_boundary_path, name="geojson").add_to(m)

# Add traffic

folium.GeoJson(aadt,
               style_function = lambda feature: style_aadt(feature)).add_to(m)

# Add Volatile Organic Compounds

vocs.apply(plot_voc, axis = 1)

m

In [8]:
# Save Map

m.save(os.path.join('.', 'MVP.html'))